# Project 1
## The Net Wars: Attack of the Bots
### Advanced Machine Learning - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# Just to make plots look better
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['axes.grid'] = True
plt.style.use('fivethirtyeight')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['lines.linewidth'] = 3

|Class  |Label |
| ----- | ---- | 
|Normal	|0     |
|Dos	|1     |
|R2L	|2     |
|U2R	|3     |
|Probe	|4     |

In [30]:
PATH_TO_DATA = 'dataset/'

test_students = pd.read_csv(PATH_TO_DATA + 'test_students.csv')
SampleID = test_students['SampleID']

train_students = pd.read_csv(PATH_TO_DATA + 'train_students.csv')


In [31]:
test_students = test_students.drop('SampleID', axis=1)

### Check Nulls and DataType

In [32]:
print(test_students.info())
print(train_students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44556 entries, 0 to 44555
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     44556 non-null  int64  
 1   protocol_type                44556 non-null  object 
 2   service                      44556 non-null  object 
 3   flag                         44556 non-null  object 
 4   src_bytes                    44556 non-null  int64  
 5   dst_bytes                    44556 non-null  int64  
 6   land                         44556 non-null  int64  
 7   wrong_fragment               44556 non-null  int64  
 8   urgent                       44556 non-null  int64  
 9   hot                          44556 non-null  int64  
 10  num_failed_logins            44556 non-null  int64  
 11  logged_in                    44556 non-null  int64  
 12  num_compromised              44556 non-null  int64  
 13  root_shell      

In [33]:
train_students.replace('normal', 0, inplace=True)
train_students.replace('Dos', 1, inplace=True)
train_students.replace('R2L', 2, inplace=True)
train_students.replace('U2R', 3, inplace=True)
train_students.replace('Probe', 4, inplace=True)


In [34]:
train_students['attack_type'].value_counts(normalize = True)

0    0.518964
1    0.358846
4    0.095545
2    0.025808
3    0.000837
Name: attack_type, dtype: float64

### Categorical

In [35]:
#Service não contêm o mesmo núemero de valores
train_students = train_students.drop('service', axis=1)
test_students = test_students.drop('service', axis=1)

train_students = pd.get_dummies(train_students, columns=['protocol_type'], drop_first=True)
#train_students = train_students.drop('service', axis=1)
#train_students = pd.get_dummies(train_students, columns=['service'], drop_first=True)
train_students= pd.get_dummies(train_students, columns=['flag'], drop_first=True)

test_students = pd.get_dummies(test_students, columns=['protocol_type'], drop_first=True)
#test_students = test_students.drop('service', axis=1)
#test_students = pd.get_dummies(test_students, columns=['service'], drop_first=True)
test_students= pd.get_dummies(test_students, columns=['flag'], drop_first=True)


In [36]:
y = train_students['attack_type']
X = train_students.drop(columns=['attack_type'], axis=1)

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
over_X, over_y = oversample.fit_resample(X, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.2, stratify=over_y, random_state=42)

classifier = Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
 
#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))
 
# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(over_X_train, over_y_train, batch_size=10 , epochs=10, verbose=1)


Epoch 1/10
21581/21581 [==============================] - 25s 1ms/step - loss: 4.4533 - accuracy: 0.8581
Epoch 2/10
21581/21581 [==============================] - 23s 1ms/step - loss: 0.9377 - accuracy: 0.8981
Epoch 3/10
21581/21581 [==============================] - 23s 1ms/step - loss: 0.2797 - accuracy: 0.9145
Epoch 4/10
21581/21581 [==============================] - 26s 1ms/step - loss: 0.2628 - accuracy: 0.9202
Epoch 5/10
21581/21581 [==============================] - 38s 2ms/step - loss: 0.2563 - accuracy: 0.9229
Epoch 6/10
21581/21581 [==============================] - 55s 3ms/step - loss: 0.2493 - accuracy: 0.9281
Epoch 7/10
21581/21581 [==============================] - 26s 1ms/step - loss: 0.3898 - accuracy: 0.9326
Epoch 8/10
21581/21581 [==============================] - 39s 2ms/step - loss: 0.8683 - accuracy: 0.9317
Epoch 9/10
21581/21581 [==============================] - 25s 1ms/step - loss: 0.2348 - accuracy: 0.9324
Epoch 10/10
21581/21581 [==============================

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

classifier.evaluate(X_test, y_test)

650/650 [==============================] - 1s 1ms/step - loss: 0.1915 - accuracy: 0.9247


[0.19150467216968536, 0.9246861934661865]

In [39]:
preds = classifier.predict(test_students)

1393/1393 [==============================] - 1s 957us/step


In [40]:
predict_test = np.argmax(preds, axis=1)

In [41]:
submission = pd.DataFrame({'SampleID': SampleID, 'Class': predict_test})
print(submission)
submission.to_csv('submission_ann.csv', index=False)

       SampleID  Class
0             0      0
1             1      0
2             2      0
3             3      0
4             4      1
...         ...    ...
44551     44551      0
44552     44552      1
44553     44553      1
44554     44554      0
44555     44555      0

[44556 rows x 2 columns]


### Todos os modelos